In [1]:
%%time 
import numpy as np # amazon dataset
from numpy.random import RandomState
import scipy.sparse as sparse
from dgl.data import AmazonCoBuyPhotoDataset


dataset = AmazonCoBuyPhotoDataset()
graph = dataset[0]  
num_class = dataset.num_classes
label = graph.ndata['label']

from networkx.convert_matrix import to_numpy_array
from dgl import to_networkx


adj = to_networkx(graph)
adj = to_numpy_array(adj)

rowsum = adj.sum(axis=1)
for i in range(adj.shape[0]):
    if rowsum[i]==0:
        rowsum[i]=1
        adj[i][i]=1
T = adj/rowsum.reshape(adj.shape[0],1)
T = np.nan_to_num(T, copy=True, nan=0.0, posinf=None, neginf=None)
Tsparse = sparse.coo_matrix(T)
T_row = Tsparse.row
T_col = Tsparse.col
T_data = Tsparse.data

Using backend: pytorch


Wall time: 59.6 s


In [2]:
# SVD Tree Construction
#inputs: T (adj), Levels, precision
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd
import numpy as np
def svdtree(T, Levels, truncpercent):
    N = T.shape[0]
    dim=int(np.ceil(N*truncpercent))
  
    philist = [] #this is U
    svlist = [] #this is S*V.T
    tlist = []
    extbases = []
    thresh = 1e-10
    T_current = T
    for j in range(Levels):
        if j>0:
            dim = int(np.ceil(tlist[j-1].shape[0]*truncpercent))
        

        u, s, vt = randomized_svd(T_current, 
                              n_components=dim,
                              n_iter=5)
        s=np.diag(s)

        philist.append(u)
        svlist.append(s @ vt)
        if j==0:
            extbases.append(u)
        else:
            extbases.append(extbases[j-1] @ u)
        T_current = u.T @ T_current

        T_current = T_current @ u

        tlist.append(T_current)
    return philist, svlist, tlist, extbases;

In [12]:
 %%time 
import scipy.sparse as sparse
Tsparse = sparse.csr_matrix(T)
truncpercent=0.5
lev=5
lev2=5 
thresh = 1e-10
N = T.shape[0]
philist, svlist, tlist, extbases = svdtree(Tsparse, lev, truncpercent)

G = sparse.eye(T.shape[0],T.shape[1])+Tsparse

for k in range(lev):
    sz = tlist[k].shape
    tempmat = extbases[k] @ (sparse.eye(sz[0],sz[1])+tlist[k])
    tempmat = tempmat @ extbases[k].T
    G = tempmat @ G 
    
GTree = svdtree(N*G, lev2, truncpercent)
g_ext_bases=GTree[3]
g_ext_op=GTree[1]
V=g_ext_bases[lev2-1] @ g_ext_op[lev2-1]
V = sparse.coo_matrix(V)
row = V.row
col = V.col
data = V.data





Wall time: 21.9 s


In [19]:
# Calculate F1 macro score by using k-nearest neighbors to classify each node.
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(V, label.reshape((label.shape[0],)))
y_pred = neigh.predict(V)
f1_score(label.reshape((label.shape[0],)), y_pred, average='macro')

0.8254762931752203